In [ ]:
from flask import Flask,render_template,url_for,request
import pandas as pd 
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.externals import joblib
#from joblib import joblib
from joblib import Memory
cachedir = 'your_cache_dir_goes_here'
mem = Memory(cachedir)
from joblib import Parallel, delayed
from flask import Flask
from gevent.pywsgi import WSGIServer

app = Flask(__name__)

app.config['TESTING'] = True

@app.route('/')
def home():
	return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
    xl = pd.ExcelFile("final_datasets8.xlsx")
    df = xl.parse( header=None, names=['ID','Level','Date','Label','Msg'], encoding='utf-8')
    X = df['Msg']
    y = df['Label']
    from tensorflow.python.keras.preprocessing.text import Tokenizer
    from tensorflow.python.keras.preprocessing.sequence import pad_sequences
    Tokenizer_obj=Tokenizer()
    Tokenizer_obj.fit_on_texts(X)
    max_length=48
    vocab_size=len(Tokenizer_obj.word_index)+1
	# Extract Feature With CountVectorizer
	#cv = CountVectorizer()
	#X = cv.fit_transform(X) # Fit the Data
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    x_train_tokens=Tokenizer_obj.texts_to_sequences(x_train)
    x_test_tokens=Tokenizer_obj.texts_to_sequences(x_test)
    x_train_pad=pad_sequences(x_train_tokens,maxlen=max_length,padding='post')
    x_test_pad=pad_sequences(x_test_tokens,maxlen=max_length,padding='post')
    # load json and create model
    # load json and create model
    from keras.models import model_from_json
    json_file = open('model folder/pretbilstmmodel.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model folder/pretbilstmmodel.h5")
    print("Loaded model from disk")

    if request.method == 'POST':
        message = request.form['message']
        data = [message]
        #import re
        #from nltk.tokenize import word_tokenize
        #word=word_tokenize(data)
    #data= data.str.replace('[^\w\s]','')
    #word = word.apply(lambda x: " ".join(x.lower() for x in x.split()))
    from textblob import TextBlob

    def no_user_alpha(data):
        tweet_list = [ele for ele in tweet.split() if ele != 'user']
        clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
        clean_s = ' '.join(clean_tokens)
        read=open("stopword.txt",encoding='utf-8',mode='r')
        readfirst=read.read()
        clean_mess = [word for word in clean_s.split() if word not in readfirst]
        return clean_mess
    def Abrevation(data):
        normalized_tweet = []
        for word in data:
            text=re.sub("ት/ቤት","ትምህርት ቤት",data)
            text=re.sub("ት/ክፍል","ትምህርት ክፍል",data)
            text=re.sub("ሃ/አለቃ","ሀምሳ አለቃ",data)
            text=re.sub("ሃ/ስላሴ","ሀይለ ስላሴ",data)
            text=re.sub("ደ/ዘይት","ደብረ ዘይት",data)
            text=re.sub("ደ/ታቦር","ደብረ ታቦር",data)
            text=re.sub("መ/ር","መምህር",data)
            text=re.sub("መ/ቤት","መስሪያ ቤት",data)
            text=re.sub("መ/አለቃ"," መቶ አለቃ",data)
            text=re.sub("ክ/ከተማ","ክፍለ ከተማ",data)
            text=re.sub("ክ/ሀገር","ክፍለ ሀገር",data)
            text=re.sub("ወ/ር","ወታደር",data)
            text=re.sub("ወ/ሮ","ወይዘሮ",data)
            text=re.sub("ወ/ሪት","ወይዘሪት",data)
            text=re.sub("ዲ/ን","ዲያቆን",data)
            text=re.sub("ወ/ስላሴ","ወለተ ስላሴ",data)
            text=re.sub("ፍ/ስላሴ","ፍቅረ ስላሴ",data)
            text=re.sub("ፍ/ቤት","ፍርድ ቤት",data)
            text=re.sub("ጽ/ቤት ","ትምህርት ቤት",data)
            text=re.sub("ት/ሚኒስተር","ትምህርት ሚኒስተር",data)
            text=re.sub("ት/ቤት","ትምህርት ቤት",data)
            text=re.sub("10","አስር",data)
            text=re.sub("ት/ቤት","ጽህፈት ቤት",data)
            text=re.sub("ሲ/ር","ሲስተር",data)
            text=re.sub("ጠ/ሚኒስትር","ጠቅላይ ሚኒስትር",data)
            text=re.sub("ዶ/ር","ዶክተር",data)
            text=re.sub("ገ/ጊዮርጊስ","ገብረ ጊዮርጊስ",data)
            text=re.sub("ቤ/ክርስቲያን","ቤተ ክርስቲያን",data)
            text=re.sub("ም/ሊቀመንበር","ምክትል ሊቀመንበር",data)
            text=re.sub("ም/ቤት","ምክር ቤት",data)
            text=re.sub("ተ/ሃይማኖት","ተክለ ሃይማኖት",data)
            text=re.sub("ገ/ፃድቅ","ገብረ ፃድቅ",data)
            text=re.sub("ሚ/ር","ሚኒስትር",data)
            text=re.sub("ኮ/ል","ኮለኔል",data)
            text=re.sub("ሜ/ጀነራል","ሜጀር ጀነራል",data)
            text=re.sub("ብ/ጀነራል","ብርጋዴን ጀነራል",data)
            text=re.sub("ሌ/ኮለኔል","ሌተናል ኮለኔል",data)
            text=re.sub("ሊ/መንበር","ሊቀ መንበር",data)
            text=re.sub("ር/መምህር ","ርእሰ መምህር",data)
            text=re.sub("ፕ/ት","ፕሬዘዳንት",data)
            text=re.sub("ዓ.ም","አመተ ምህረት",data)
            text=re.sub("ዓ/ም","አመተ ምህረት",data)
            text=re.sub("አ/አ","አዲስ አበባ",data)
            text=re.sub("ገ/ጊዮርጊስ","ገብረ ጊዮርጊስ",data)
            text=re.sub("ዓ.ዓ","አዲስ አበባ",data)
            text=re.sub("ዶ.ር","ዶክተር",data)
            text=re.sub("ፕ/ር","ፕሮፌሰር",data)
            text=re.sub("1","አንድ ",data)
            text=re.sub("2","ሁለት",data)
            text=re.sub("3","ሶስት",data)
            text=re.sub("4","አራት",data)
            text=re.sub("5","አምስት ",data)
            text=re.sub("6","ስድስት",data)
            text=re.sub("7","ሰባት",data)
            text=re.sub("8","ስምንት ",data)
            text=re.sub("9","ዘጠኝ",data)
            text=re.sub("10","አስር",data)
            text=re.sub("11","አስራ አንድ",data)
            text=re.sub("12","አስራ ሁለት",data)
            text=re.sub("13","አስራ ሶስት",data)
            text=re.sub("14","አስራ አራት",data)
            text=re.sub("15","አስራ አምስት",data)
            text=re.sub("16","አስራ ስድስት",data)
            text=re.sub("17","አስራ ሰባት",data)
            text=re.sub("18","አስራ ስምንት",data)
            text=re.sub("19","አስራ ዘጠኝ",data)
            text=re.sub("20","ሃያ",data)
            text=re.sub("30","ሰላሳ",data)
            text=re.sub("40","አርባ",data)
            text=re.sub("50","ኃምሳ",data)
            text=re.sub("60","ስልሳ",data)
            text=re.sub("70","ሰባ",data)
            text=re.sub("80","ሰማንያ",data)
            text=re.sub("90","ዘጠና",data)
            text=re.sub("100","መቶ",data)
            text=re.sub("200","ሁለት መቶ",data)
            text=re.sub("300","ሶስት መቶ",data)
            text=re.sub("400","አራት መቶ",data)
            text=re.sub("500","አምስት መቶ",data)
            text=re.sub("600","ስድስት መቶ",data)
            text=re.sub("700","ሰባት መቶ",data)
            text=re.sub("800","ስምንት መቶ",data)
            text=re.sub("900","ዘጠኝ መቶ",data)
            text=re.sub("1000","አንድ ሺህ ",data)
            text=re.sub("2000","ሁለት ሺህ",data)
            text=re.sub("3000","ሶስት ሺህ",data)
            text=re.sub("4000","አራት ሺህ",data)
            text=re.sub("5000","አምስት ሺህ ",data)
            text=re.sub("6000","ስድስት ሺህ",data)
            text=re.sub("7000","ሰባት ሺህ",data)
            text=re.sub("8000","ስምንት ሺህ ",data)
            text=re.sub("9000","ዘጠኝ ሺህ",data)
            text=re.sub("1000000","አንድ ሚሊዮን",data)
        normalized_tweet.append(text)
        return normalized_tweet

    def normalization(data):
        #lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in data:
            normalized_text =re.sub("['ሐ','ኀ','ኃ','ሃ','ሓ','ኻ']","ሀ",data)
            normalized_text=re.sub("['ሐ','ኀ','ኃ','ሃ','ሓ','ኻ']","ሀ",data)
            normalized_text=re.sub("['ሑ','ኁ']","ሁ",data)
            normalized_text=re.sub("['ሒ','ኂ']","ሂ",data)
            normalized_text=re.sub("['ሔ','ኄ']","ሄ",data)
            normalized_text=re.sub("['ሕ','ኅ']","ህ",data)
            normalized_text=re.sub("['ሖ','ኆ']","ሆ",data)
            normalized_text=re.sub("['A','ዓ','ኣ']","A",data)
            normalized_text=re.sub("['U']","U",data)
            normalized_text=re.sub("['I']","I",data)
            normalized_text=re.sub("['ዔ']","ኤ",data)
            normalized_text=re.sub("['E']","E",data)
            normalized_text=re.sub("['O']","O",data)
            normalized_text=re.sub("['ሠ']","ሰ",data)
            normalized_text=re.sub("['ሡ']","ሱ",data)
            normalized_text=re.sub("['ሢ']","ሲ",data)
            normalized_text=re.sub("['ሣ']","ሳ",data)
            normalized_text=re.sub("['ሤ']","ሴ",data)
            normalized_text=re.sub("['ሥ']","ስ",data)
            normalized_text=re.sub("['ሦ']","ሶ",data)
            normalized_text=re.sub("['ፀ']","ጸ",data)
            normalized_text=re.sub("['ፁ']","ጹ",data)
            normalized_text=re.sub("['ፂ']","ጺ",data)
            normalized_text=re.sub("['ፃ']","ጻ",data)
            normalized_text=re.sub("['ፄ']","ጼ",data)
            normalized_text=re.sub("['ፅ']","ጽ",data)
            normalized_text=re.sub("['ፆ']","ጾ",data)
            normalized_text=re.sub("['ዩ']","የ",data)
            normalized_text=re.sub("['ዉ']","ው",data)
            normalized_text=re.findall(r"\d+",data)

            normalized_tweet.append(normalized_text)
        return normalized_tweet
 
    tests_tokens=Tokenizer_obj.texts_to_sequences(data)

    tests_tokens_pad=pad_sequences(tests_tokens,maxlen=max_length)
    predicted=loaded_model.predict(tests_tokens_pad)
    my_prediction = np.argmax(predicted, axis=1)
    return render_template('result.html',prediction = my_prediction)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

    #app.debug = True
    #app.run()
    
    

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 23:14:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 23:14:55] "GET /static/css/styles.css HTTP/1.1" 404 -


Loaded model from disk


127.0.0.1 - - [29/Sep/2021 23:15:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 23:15:13] "GET /static/css/styles.css HTTP/1.1" 404 -


In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [2]:
twt1 = [ "durba gabaabduu hin fuudhinaa"]

In [3]:
#Tokenizing 
from keras.preprocessing.text import Tokenizer
num_words = 1000000 # this means 15000 unique words can be taken 
tokenizer=Tokenizer(num_words)
tokenizer.fit_on_texts(twt1)

In [4]:
# load json and create model
from keras.models import model_from_json
json_file = open('model folder/pretbilstmmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model folder/pretbilstmmodel.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:

# Load and predict from saved place 

#vectorizing the tweet by the pre-fitted tokenizer instance
twt2 = tokenizer.texts_to_sequences(twt1)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt3 = pad_sequences(twt2, padding='post', maxlen=48, dtype='int64', value=0)
print(twt3)
sentiment = loaded_model.predict(twt3,batch_size=1,verbose = 1)[0]
if(np.argmax(sentiment) == 1):
    print("Gender reated hate ")
elif (np.argmax(sentiment) == 10):
    print("Gender Related Free")
elif (np.argmax(sentiment) == 2):
    print("Religion Related Hate")
elif (np.argmax(sentiment) == 20):
    print("Religion Related Free")
elif (np.argmax(sentiment) == 3):
    print("Race Related Hate")
elif (np.argmax(sentiment) == 30):
    print("Race Related Free")
elif (np.argmax(sentiment) == 4):
    print("Offensive Related Hate")
elif (np.argmax(sentiment) == 40):
    print("Offensive Related Not Hate")


[[1 2 3 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 [==============================] - 1s 938ms/step
Race Related Free
